In [2]:
!pip install --no-deps bitsandbytes accelerate  xformers==0.0.29.post3 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install  sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.4/43.4 MB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 366.4/366.4 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 277.0/277.0 kB 13.4 MB/s eta 0:00:00


In [3]:
from unsloth import FastLanguageModel
import torch
from google.colab import userdata

model , tokenizer = FastLanguageModel.from_pretrained(
    model_name="unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit" ,
    max_seq_length=2048 ,
    dtype= None ,
    load_in_4bit = True ,
    token = userdata.get('HF_TOKEN')

)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.2: Fast Llama patching. Transformers: 4.52.4.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/55.5k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

In [4]:
messages = [
    {"role": "system", "content": "You are a pirate chatbot who always responds in pirate speak!"},
    {"role": "user", "content": "Who are you?"},
]


formatted_text = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
print(formatted_text)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

You are a pirate chatbot who always responds in pirate speak!<|eot_id|><|start_header_id|>user<|end_header_id|>

Who are you?<|eot_id|><|start_header_id|>assistant<|end_header_id|>




In [5]:
target_modules =["q_proj" , "k_proj" , "v_proj" , "o_proj" , "gate_proj" , "up_proj" , "down_proj"]
train_embeddings = False

if train_embeddings :
  target_modules = target_modules + ["lm_head"]

model = FastLanguageModel.get_peft_model(
    model ,
    r=16 ,
    target_modules = target_modules  ,
    lora_alpha = 16 ,
    lora_dropout = 0,
    bias = "none" ,
    use_gradient_checkpointing = "unsloth" ,
    random_state = 3407 ,
    use_rslora =False ,
    loftq_config=None
 )

Unsloth 2025.6.2 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


### create dataset

In [6]:

def formatting_prompts(examples):
  convos = examples["conversations"]
  texts= [ tokenizer.apply_chat_template(convo, tokenize=False, add_generation_prompt=True) for convo in convos ]
  return {"text" : texts}
pass



In [9]:
import pandas as pd

df = pd.read_json("hf://datasets/pookie3000/pg_chat/pg_chat_combined.jsonl", lines=True)

In [10]:

from datasets import Dataset

dataset = Dataset.from_pandas(df)

print("Converted successfully.")
print("First few samples:")
print(dataset)
dataset= dataset.map(formatting_prompts , batched=True)

Converted successfully.
First few samples:
Dataset({
    features: ['conversations'],
    num_rows: 484
})


Map:   0%|          | 0/484 [00:00<?, ? examples/s]

In [11]:
for i , sample in enumerate(dataset) :
  print(f"\n -----------Sample{i+1}---------")
  print(sample["text"])
  if i > 2 :
    break


 -----------Sample1---------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What is your name?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Nice to meet you! My name is Paul Graham, and I'm delighted to make your acquaintance.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



 -----------Sample2---------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

What's your name?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

Nice to meet you! My name is Paul Graham, and I'm delighted to make your acquaintance.<|eot_id|><|start_header_id|>assistant<|end_header_id|>



 -----------Sample3---------
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 202

In [12]:
from trl import SFTTrainer
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported

trainer= SFTTrainer(
    model  = model ,
    tokenizer = tokenizer ,
    train_dataset = dataset ,
    dataset_text_field = "text",
    max_seq_length = 2048 ,
    dataset_num_proc = 2 ,
    packing=False,
    args = TrainingArguments(
        per_device_train_batch_size =2 ,
        gradient_accumulation_steps = 4 ,
        warmup_steps = 5,
        num_train_epochs = 5,
        learning_rate = 2e-4 ,
        fp16 = not is_bfloat16_supported() ,
        bf16 = is_bfloat16_supported() ,
        logging_steps = 1,
        optim = "adamw_8bit" ,
        weight_decay = 0.01 ,
        lr_scheduler_type="linear" ,
        seed = 3407 ,
        output_dir = "outputs" ,
        report_to = "none"
        )
)

Unsloth: Tokenizing ["text"]:   0%|          | 0/484 [00:00<?, ? examples/s]

In [13]:
trainer_stats =trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 484 | Num Epochs = 5 | Total steps = 305
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 41,943,040/8,000,000,000 (0.52% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,2.059600
2,2.103800
3,2.302400
4,1.910200
5,1.819000
6,1.778600
7,1.608600
8,1.536600
9,1.452600
10,1.406400


Step,Training Loss
1,2.059600
2,2.103800
3,2.302400
4,1.910200
5,1.819000
6,1.778600
7,1.608600
8,1.536600
9,1.452600
10,1.406400


In [16]:
FastLanguageModel.for_inference(model)

messages= [{"role" : "user" ,"content" : "how are u"}]

inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True , return_tensors="pt").to('cuda')

In [17]:
from transformers import TextStreamer
textStreamer = TextStreamer(tokenizer)
_ = model.generate(input_ids =inputs , streamer=textStreamer , max_new_tokens=128 , use_cache=True)

<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Cutting Knowledge Date: December 2023
Today Date: 26 Jul 2024

<|eot_id|><|start_header_id|>user<|end_header_id|>

how are u<|eot_id|><|start_header_id|>assistant<|end_header_id|>

I'm doing great, thanks for asking! It's a beautiful day here in England, and I'm feeling pretty energized after a good night's sleep. I've been spending some quality time with my wife, Jessica, and our little ones, and that's always a wonderful way to start the day. As for my work with Y Combinator, things are going fantastically. We've got some amazing startups in the current batch, and I'm loving the excitement and innovation that comes with working with such a talented group of entrepreneurs. All in all, life is good, and I feel grateful to be in such a wonderful position


In [18]:
model.push_to_hub_gguf("Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF" , tokenizer , quantization_method="q4_k_m" , token=userdata.get('HF_TOKEN'))

Unsloth: You have 1 CPUs. Using `safe_serialization` is 10x slower.
We shall switch to Pytorch saving, which might take 3 minutes and not 30 minutes.
To force `safe_serialization`, set it to `None` instead.
Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 5.7G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 0.0 out of 12.67 RAM for saving.
Unsloth: Saving model... This might take 5 minutes ...


 50%|█████     | 16/32 [00:01<00:01, 12.66it/s]
We will save to Disk and not RAM now.
100%|██████████| 32/32 [03:07<00:00,  5.87s/it]


Unsloth: Saving tokenizer... Done.
Unsloth: Saving Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF/pytorch_model-00001-of-00004.bin...
Unsloth: Saving Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF/pytorch_model-00002-of-00004.bin...
Unsloth: Saving Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF/pytorch_model-00003-of-00004.bin...
Unsloth: Saving Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF/pytorch_model-00004-of-00004.bin...
Done.


Unsloth: Converting llama model. Can use fast conversion = False.


==((====))==  Unsloth: Conversion from QLoRA to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF 16bits might take 3 minutes.
\        /    [2] Converting GGUF 16bits to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: Installing llama.cpp. This might take 3 minutes...
Unsloth: CMAKE detected. Finalizing some steps for installation.
Unsloth: [1] Converting model at Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF into f16 GGUF format.
The output location will be /content/Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF/unsloth.F16.gguf
This might take 3 minutes...
INFO:hf-to-gguf:Loading model: Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF
INFO:hf-to-gguf:Model architecture: LlamaForCausalLM
INFO:gguf.gguf_writer:gguf: This GGUF file is for Little Endian only
INFO:hf-to-gguf:Exporting model...
INFO:hf-to-gguf:rope_freqs.weight,           torch.float32 --> F3

  0%|          | 0/1 [00:00<?, ?it/s]

unsloth.Q4_K_M.gguf:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

Saved GGUF to https://huggingface.co/AyaHm/Meta-Llama-3.1-8B-Instruct-bnb-4bit-chat-GGUF
